In [1]:
cd(@__DIR__)
ENV["CELLLISTMAP_8.3_WARNING"] = "false"
include("../../src/juliaEAM.jl")
include("../../src/lammpsIO.jl")

using Pkg
Pkg.activate(".")

using Printf
using AtomsCalculators
using Unitful: Å, nm
using StaticArrays: SVector
using Molly
using LinearAlgebra
using DelimitedFiles
using UnitfulAtomic
import PeriodicTable
using ProgressMeter
using Random

  Activating new project at `~/Documents/ABCD_J/EAM/multi_defects`


## Define Aluminum

In [2]:
al_LatConst = 4.0495/10 # nm
atom_mass = 26.9815u"u"  # Atomic mass of aluminum in grams per mole

## `AtomCalculators` force/energy calculator and `Molly` simulator
eam = EAM()
fname = "Al99.eam.alloy"
read_potential!(eam, fname)

### Define customized interaction type in `AtomsCalculators`
struct EAMInteractionJulia
    calculator::Any  # Holds the ASE EAM calculator reference
    f_energy::Any    # Holds the energy function
    f_forces::Any    # Holds the forces function
    f_atomstress::Any  # Holds the atomic level stresses function
end

In [3]:
include("../../src/simulator.jl")

eamJulia = EAMInteractionJulia(eam,calculate_energy,calculate_forces,calculate_atomstress)

EAMInteractionJulia(EAM(1, ["Al"], 10000, 0.0002, 10000, 0.000628721, 6.28721, [1.042222107228152e-10 -0.002833153928160009 … -2.076421465206393 -2.076023741417596], [0.2785894267518672 0.2784103240595593 … 0.0 0.0], [13], [26.982], [4.05], "fcc", [0.0;;; 0.03937793713511766;;; 0.07866615574588072;;; … ;;; -9.062527475633544e-10;;; -2.684334141141742e-10;;; -3.337655546827989e-11], 0.0:0.000628721:6.286581279000001, 0.0:0.0002:1.9998, [0.0, 0.000628721, 0.001257442, 0.0018861630000000002, 0.002514884, 0.0031436050000000003, 0.0037723260000000003, 0.004401047, 0.005029768, 0.005658489000000001  …  6.280922790000001, 6.281551511000001, 6.282180232000001, 6.282808953000001, 6.283437674000001, 6.284066395000001, 6.284695116000001, 6.285323837000001, 6.285952558000001, 6.286581279000001], [0.0, 0.0002, 0.0004, 0.0006, 0.0008, 0.001, 0.0012, 0.0014, 0.0016, 0.0018  …  1.998, 1.9982, 1.9984, 1.9986, 1.9988, 1.999, 1.9992, 1.9994, 1.9996, 1.9998], Interpolations.Extrapolation{Float64, 1, Inter

In [4]:
function initialize_system_dump(;loggers=(coords=CoordinateLogger(1),),filename_dump="")
    n_atoms, box_size, coords_molly, attypes = lmpDumpReader(filename_dump)
    molly_atoms = [Molly.Atom(index=i, charge=0, mass=atom_mass, 
                    #   σ=2.0u"Å" |> x -> uconvert(u"nm", x), ϵ=ϵ_kJ_per_mol
                    ) for i in 1:length(coords_molly)]
    # Specify boundary condition
    boundary_condition = Molly.CubicBoundary(box_size[1],box_size[2],box_size[3])

    atom_positions_init = copy(coords_molly)
    molly_atoms_init = copy(molly_atoms)

    DNF = DistanceNeighborFinder(
        eligible=trues(length(molly_atoms_init), length(molly_atoms_init)),
        n_steps=1e3,
        dist_cutoff=8u"Å")
    TNF = TreeNeighborFinder(
        eligible=trues(length(molly_atoms_init), length(molly_atoms_init)), 
        n_steps=1e3,
        dist_cutoff=8u"Å")

    # Initialize the system with the initial positions and velocities
    system_init = Molly.System(
    atoms=molly_atoms_init,
    atoms_data = [AtomData(element="Al", atom_type=string(attypes[ia])) for (ia,a) in enumerate(molly_atoms_init)],
    coords=atom_positions_init,  # Ensure these are SVector with correct units
    boundary=boundary_condition,
    # loggers=Dict(:kinetic_eng => Molly.KineticEnergyLogger(100), :pot_eng => Molly.PotentialEnergyLogger(100)),
    neighbor_finder = DNF,
    loggers=loggers,
    energy_units=u"eV",  # Ensure these units are correctly specified
    force_units=u"eV/Å"  # Ensure these units are correctly specified
    )
    return system_init
end

initialize_system_dump (generic function with 1 method)

## Initialize sys

In [5]:
filename_dump = "./out_vacancy_2.dump"
molly_system = initialize_system_dump(filename_dump = filename_dump)
neighbors_all = get_neighbors_all(molly_system)

2046-element Vector{Vector{Int64}}:
 [263, 265, 266, 267, 268, 269, 270, 271, 272, 283  …  1552, 1632, 1633, 1635, 1648, 1649, 1650, 1651, 1664, 1776]
 [3, 4, 5, 6, 7, 8, 9, 12, 17, 18  …  1933, 1934, 1935, 1948, 1949, 1950, 1951, 1966, 2045, 2046]
 [2, 4, 5, 6, 7, 8, 9, 12, 17, 18  …  1279, 1293, 1405, 1600, 1933, 1934, 1935, 1950, 2045, 2046]
 [2, 3, 5, 6, 7, 8, 17, 18, 19, 20  …  1934, 1935, 1946, 1948, 1949, 1950, 1951, 1966, 2045, 2046]
 [2, 3, 4, 6, 7, 8, 9, 10, 11, 12  …  1054, 1149, 1150, 1165, 1277, 1933, 1934, 1950, 2045, 2046]
 [2, 3, 4, 5, 7, 8, 9, 10, 11, 12  …  1011, 1012, 1013, 1014, 1015, 1016, 1017, 1018, 1019, 1023]
 [2, 3, 4, 5, 6, 8, 9, 10, 11, 12  …  1011, 1012, 1013, 1014, 1015, 1016, 1017, 1018, 1019, 1023]
 [2, 3, 4, 5, 6, 7, 9, 10, 11, 12  …  1053, 1054, 1149, 1165, 1166, 1181, 1182, 1277, 1934, 1950]
 [2, 3, 5, 6, 7, 8, 10, 11, 12, 13  …  1014, 1015, 1016, 1017, 1018, 1019, 1020, 1021, 1022, 1023]
 [5, 6, 7, 8, 9, 11, 12, 13, 14, 15  …  1040, 1043, 1139, 1152,

energy minimization

In [6]:
simulator = ABCSimulator(sigma=1.0*u"Å", W=1.0*u"eV", max_steps=1, max_steps_minimize=20, step_size_minimize=5e-3u"ps", tol=1e-6u"eV/Å")
Minimize_FIRE!(molly_system, simulator, eamJulia, nothing, neighbors_all;
         n_threads=1, frozen_atoms=[], neig_interval=5, print_nsteps=true,
         mass=26.9815u"u")
neighbors_all = get_neighbors_all(molly_system)

2046-element Vector{Vector{Int64}}:
 [263, 265, 266, 267, 268, 269, 270, 271, 272, 283  …  1552, 1632, 1633, 1635, 1648, 1649, 1650, 1651, 1664, 1776]
 [3, 4, 5, 6, 7, 8, 9, 12, 17, 18  …  1933, 1934, 1935, 1948, 1949, 1950, 1951, 1966, 2045, 2046]
 [2, 4, 5, 6, 7, 8, 9, 12, 17, 18  …  1279, 1293, 1405, 1600, 1933, 1934, 1935, 1950, 2045, 2046]
 [2, 3, 5, 6, 7, 8, 17, 18, 19, 20  …  1934, 1935, 1946, 1948, 1949, 1950, 1951, 1966, 2045, 2046]
 [2, 3, 4, 6, 7, 8, 9, 10, 11, 12  …  1054, 1149, 1150, 1165, 1277, 1933, 1934, 1950, 2045, 2046]
 [2, 3, 4, 5, 7, 8, 9, 10, 11, 12  …  1011, 1012, 1013, 1014, 1015, 1016, 1017, 1018, 1019, 1023]
 [2, 3, 4, 5, 6, 8, 9, 10, 11, 12  …  1011, 1012, 1013, 1014, 1015, 1016, 1017, 1018, 1019, 1023]
 [2, 3, 4, 5, 6, 7, 9, 10, 11, 12  …  1053, 1054, 1149, 1165, 1166, 1181, 1182, 1277, 1934, 1950]
 [2, 3, 5, 6, 7, 8, 10, 11, 12, 13  …  1014, 1015, 1016, 1017, 1018, 1019, 1020, 1021, 1022, 1023]
 [5, 6, 7, 8, 9, 11, 12, 13, 14, 15  …  1040, 1043, 1139, 1152,

run ABC

In [7]:
frozen_atoms = []
nopenalty_atoms = []
N_free = length(molly_system.coords)-length(nopenalty_atoms)
print("exclude ",length(nopenalty_atoms)," atoms from E_phi calculation\n")
print("using ",N_free," atoms for E_phi calculation\n")

# sigma = sqrt(0.006*3*N_free)
sigma = sqrt(0.27)
W = 0.1
@printf("sigma^2 = %e, %e Å/dof^1/2\n W = %e eV\n",ustrip(sigma^2), ustrip(sigma/sqrt(3*N_free)),ustrip(W))

simulator = ABCSimulator(sigma=sigma*u"Å", W=W*u"eV", 
                         max_steps=20000, max_steps_minimize=30, step_size_minimize=5e-3u"ps", tol=1e-3u"eV/Å")

simulate!(molly_system, simulator, eamJulia, n_threads=1, 
         #   fname="output_stress_cube8.txt", fname_dump="stress_cube8.dump", fname_min_dump="min_stress_cube8.dump",
         fname="test.txt", fname_dump="test.dump", fname_min_dump="test.dump", # for speed test
         neig_interval=30, loggers_interval=10, dump_interval=100, start_dump=0,
         minimize_only=false, 
         d_boost=1e-6u"Å", 
         frozen_atoms=frozen_atoms, nopenalty_atoms=nopenalty_atoms, 
         p_drop = 1-1/32, p_keep=0, n_memory=0, n_search=100,
         p_stress = 1-12/2047)

exclude 0 atoms from E_phi calculation
using 2046 atoms for E_phi calculation
sigma^2 = 2.700000e-01, 6.632365e-03 Å/dof^1/2
 W = 1.000000e-01 eV
step 0: -6873.2102638120095 eV


Progress: 100%|█████████████████████████████████████████| Time: 6:47:02


System with 2046 atoms, boundary CubicBoundary{Quantity{Float64, 𝐋, Unitful.FreeUnits{(Å,), 𝐋, nothing}}}(Quantity{Float64, 𝐋, Unitful.FreeUnits{(Å,), 𝐋, nothing}}[32.39491355457103 Å, 32.39491355457103 Å, 32.39491355457103 Å])